# Use a large language model to play the role of devil's advocate

This notebook demonstrates how to:
- Read proposed plans from a mural
- For each plan, prompt a large language model to generate arguments against the plan
- Post the arguments in the mural

Notebook sections:

**Section A - Explore prompting large languge models**
- [Step 1: Set up IBM watsonx.ai foundation model Python library prerequisites](#step1)
- [Step 2: Create a function for prompting a model to generate arguments](#step2)

**Section B - Set up your mural**
- [Step 3: Set up MURAL prerequisites](#step3)
- [Step 4: Read plans from your mural](#step4)

**Section C - Generate and post arguments**
- [Step 5: Generate arguments against the plans](#step5)
- [Step 6: Post arguments into the mural](#step6)

When you run this notebook, your mural will look something like the following image:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-13_devils-advocate_02.png" width="50%" title="Image of a mural" />

# Section A - Explore prompting large languge models

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-13_devils-advocate_05.png" width="50%" title="Image of a mural" />

<a id="step1"></a>
## Step 1: Set up IBM watsonx.ai foundation model Python library prerequisites
Before you can prompt a foundation model in watsonx.ai, you must perform the following setup tasks:
- 1.1 Create an instance of the Watson Machine Learning service
- 1.2 Associate the Watson Machine Learning instance with the current project
- 1.3 Create an IBM Cloud API key
- 1.4 Create a credentials dictionary for Watson Machine learning
- 1.5 Look up the current project ID

### 1.1 Create an instance of the Watson Machine Learning service
If you don't already have an instance of the IBM Watson Machine Learning service, you can create an instance of the service from the IBM Cloud catalog: <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank">Watson Machine Learning service</a>

### 1.2 Associate an instance of the Watson Machine Learning service with the current project
The _current project_ is the project in which you are running this notebook.

If an instance of Watson Machine Learning is not already associated with the current project, follow the instructions in this topic to do so: <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html?context=wx&audience=wdp" target="_blank">Adding associated services to a project</a>

### 1.3 Create an IBM Cloud API key
Create an IBM Cloud API key by following these instruction: <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui#create_user_key" target="_blank">Creating an IBM Cloud API key</a>

Then paste your new IBM Cloud API key in the code cell below.

In [1]:
g_cloud_apikey = ""

### 1.4 Create a credentials dictionary for Watson Machine learning
See: [Authentication](https://ibm.github.io/watson-machine-learning-sdk/setup_cloud.html#authentication)

In [2]:
# See: https://ibm.github.io/watson-machine-learning-sdk/setup_cloud.html#authentication
g_wml_credentials = { 
    "url"    : "https://us-south.ml.cloud.ibm.com", 
    "apikey" : g_cloud_apikey
}

### 1.5 Look up the current project ID
The _current project_ is the project in which you are running this notebook.  You can get the ID of the current project programmatically by running the following cell.

In [3]:
import os

g_project_id = os.environ["PROJECT_ID"]

<a id="step2"></a>
## Step 2: Create a function for prompting a model to generate arguments
- 2.1 Experiment in Prompt Lab
- 2.2 Specify your selected model ID, prompt parameters, and prompt text template
- 2.2 Define a function to generate a list of arguments against a given plan

### 2.1 Experiment in Prompt Lab 
The Prompt Lab in watsonx.ai is a graphical interface for experimenting with prompting foundation models.

Experiment in Prompt Lab to discover what works best:
- Which model returns ideal results
- What parameter settings (eg. decoding) work best
- What prompt text causes the model to respond the way you want

See: [Prompt Lab](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-prompt-lab.html?context=wx&audience=wdp)

### 2.2 Specify your selected model_id, prompt parameters, and prompt text template
In the following three cells, there are example model ID, prompt parameters, and prompt text template you can use.

Replace any of these with values you discovered while experimenting in Prompt Lab.

In [4]:
# Example models:
#
# google/flan-ul2
# google/flan-t5-xxl
# bigscience/mt0-xxl
# eleutherai/gpt-neox-20b
# ibm/granite-13b-instruct-v1

g_model_id = "ibm/granite-13b-instruct-v1"

In [5]:
g_prompt_parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 200,
    "min_new_tokens": 0,
    "random_seed": 1683314208,
    "stop_sequences": [ "\n\n" ],
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 1,
    "repetition_penalty": 1.0
}

**NOTE**

In the following example template, notice the use of `%s` as a placeholder for the plan description. 

If you replace this template with a prompt you discovered through your experiments in Prompt Lab, remember to include a placeholder for the plan description.

In [6]:
g_prompt_template = """You are playing the role of devil's advocate.  Argue against the proposed plan. List 4 detailed, unique, compelling reason why moving forward with the plan would be a bad choice.  Consider all types of risks. Do not repeat answers.

Plan we are considering: 
Extend our store hours.
Four problems with this plan are:
1. We'll have to pay more for staffing
2. Risk of theft increases late at night
3. Clerks might not want to work later hours
4. Changing hours could cause confusion

Plan we are considering: 
Open a second location for our business.
Four problems with this plan are:
1. Managing two locations will more than twice as time-consuming as managing just one
2. Creating a new location doesn't guarantee twice as many customers
3. Customers might be confused by there being two locations
4. Having two locations dilutes the uniqueness of the store's brand

Plan we are considering: %s
Four problems with this plan are:
"""

def createPromptText( plan_text ):
    return g_prompt_template % ( plan_text )

In [7]:
g_test_plan = "Build an addition on the store to have more room to display merchandise"
g_test_prompt_text = createPromptText( g_test_plan )
print( g_test_prompt_text )

You are playing the role of devil's advocate.  Argue against the proposed plan. List 4 detailed, unique, compelling reason why moving forward with the plan would be a bad choice.  Consider all types of risks. Do not repeat answers.

Plan we are considering: 
Extend our store hours.
Four problems with this plan are:
1. We'll have to pay more for staffing
2. Risk of theft increases late at night
3. Clerks might not want to work later hours
4. Changing hours could cause confusion

Plan we are considering: 
Open a second location for our business.
Four problems with this plan are:
1. Managing two locations will more than twice as time-consuming as managing just one
2. Creating a new location doesn't guarantee twice as many customers
3. Customers might be confused by there being two locations
4. Having two locations dilutes the uniqueness of the store's brand

Plan we are considering: Build an addition on the store to have more room to display merchandise
Four problems with this plan are:



### 2.3 Define a function to generate a list of arguments against a given plan
You can prompt foundation models in IBM watsonx.ai programmatically using the foundation models Python library.

See:
- <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-python-lib.html?context=wx&audience=wdp" target="_blank">Introduction to the foundation models Python library</a>
- <a href="https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html" target="_blank">Foundation models Python library reference</a>

Testing the library:

In [8]:
from ibm_watson_machine_learning.foundation_models import Model
import json

test_model = Model( g_model_id, g_wml_credentials, g_prompt_parameters, g_project_id )

test_response = test_model.generate( g_test_prompt_text )

print( json.dumps( test_response, indent=3 ) )

{
   "model_id": "ibm/granite-13b-instruct-v1",
   "created_at": "2023-10-02T18:55:19.273Z",
   "results": [
      {
         "generated_text": "1. It will be expensive\n2. Construction will likely interfere with business\n3. There is no guarantee that having more room to display products will result in more sales\n4. There are other ways to grow sales without more room",
         "generated_token_count": 48,
         "input_token_count": 229,
         "stop_reason": "EOS_TOKEN",
         "seed": 1683314208
      }
   ]
}


Defining a function:

In [9]:
import re

def generate( model_id, prompt_parameters, prompt_text, b_debug=False ):
    model = Model( model_id, g_wml_credentials, prompt_parameters, g_project_id )
    raw_response = model.generate( prompt_text )
    if b_debug:
        print( "\nraw_response:\n" + json.dumps( raw_response, indent=3 ) )
    if ( "results" in raw_response ) \
       and ( len( raw_response["results"] ) > 0 ) \
       and ( "generated_text" in raw_response["results"][0] ):
        return raw_response, raw_response["results"][0]["generated_text"]
    else:
        print( "\nThe model failed to generate an answer" )
        print( "\nDebug info:\n" + json.dumps( raw_response, indent=3 ) )
        return raw_response, ""

def listArgumentsAgainst( model_id, prompt_parameters, plan_text, b_debug=False ):
    prompt_text = createPromptText( plan_text )
    raw_response, generated_output = generate( model_id, prompt_parameters, prompt_text, b_debug )
    arguments_arr = re.findall( r"\d+\..*", generated_output )
    if len( arguments_arr ) < 4:
        print( "Failed to generate four arguments")
        print( "Raw model output:\n" + json.dumps( raw_response, indent=3 ) )
        return None
    for i in range( len( arguments_arr ) ):
        arguments_arr[i] = re.sub( r"^\d+\.\s+", "", arguments_arr[i] )
    return arguments_arr

In [10]:
arguments_arr = listArgumentsAgainst( g_model_id, g_prompt_parameters, g_test_plan )
if arguments_arr is not None:
    print( json.dumps( arguments_arr, indent=3 ) )

[
   "It will be expensive",
   "Construction will likely interfere with business",
   "There is no guarantee that having more room to display products will result in more sales",
   "There are other ways to grow sales without more room"
]


# Section B - Set up your mural

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-13_devils-advocate_04.png" width="50%" title="Image of a mural" />

<a id="step3"></a>
## Step 3: Set up MURAL prerequisites
- 3.1 Create empty, sample mural
- 3.2 Collect mural ID
- 3.3 Get Oauth token
- 3.4 Populate mural

### 3.1 Create empty, sample mural
In the MURAL web interface, create a new, empty mural.

### 3.2 Collect mural ID
You can find the mural ID in the url of a mural.

Mural urls look something like this:

```
https://app.mural.co/t/<workspace>/m/<workspace>/<id>/...
```

What you need to pass to the MURAL API is just after the `/m/`: the \<workspace> and the \<id>.  And you need to join then with a period.

For example, if you have a mural with this url:

```
https://app.mural.co/t/teamideas1234/m/teamideas1234/1234567890123/...
```

Then, the mural ID is: `teamideas1234.1234567890123`

In [11]:
g_mural_id = ""

### 3.3 Collect OAuth token

In [43]:
g_auth_token = ""

### 3.4 Populate mural

In [15]:
import urllib
import json

url = "https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/murals/sample-13_devils-advocate.json"
response = urllib.request.urlopen( url )
encoding = response.info().get_content_charset( "utf8" )
sample_widgets_arr = json.loads( response.read().decode( encoding ) )

In [22]:
import requests
import copy

def putWidget( auth_token, mural_id, widget ):
    # https://developers.mural.co/public/reference/createstickynote
    # https://developers.mural.co/public/reference/createtextbox
    # https://developers.mural.co/public/reference/createarrow
    widget_type = "textbox" if ( "text" == widget["type"] ) else widget["type"]
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/" + re.sub( "\s+", "-", widget_type )
    headers = { "Accept"        : "application/json", 
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    parms = copy.deepcopy( widget )
    if "id" in parms:
        del parms["id"]
    if "type" in parms:
        del parms["type"]
    #if "style" in parms:
    #    del parms["style"]
    response = requests.request( "POST", url, headers = headers, json = parms )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( "[ " + widget["id"] + " ] " + msg )
    else:
        print( "[ " + widget["id"] + " ] Success" )

In [ ]:
for widget in sample_widgets_arr:
    putWidget( g_auth_token, g_mural_id, widget )

<a id="step4"></a>
## Step 4: Read plans from your mural
- 4.1 Read widgets from the mural
- 4.2 Establish mural coordinates

### 4.1 Read widgets from the mural

In [24]:
import requests

def listWidgets( auth_token, mural_id ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets"
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    return response_json["value"]

In [25]:
g_widgets_arr = listWidgets( g_auth_token, g_mural_id )

In [ ]:
print( json.dumps( g_widgets_arr, indent=3 ) )

### 4.2 Establish mural coordinates
To be able to paste arguments against the plans in the mural, it's not enough to just make a list of the plans in the mural.  

Instead, we need to know the quadrants where the plans are located in the mural.

In [27]:
import numpy as np
import math

def getAngle( x, y, center_x, center_y ):
    z = ( x - center_x ) + 1j * ( y - center_y )
    return np.angle( z, deg = True )

def removeHTML( html ):
    txt = re.sub( r"\<[^>]+\>", " ", html )
    return txt

def removeExtraSpaces( txt ):
    txt = re.sub( r"^\s+", "", txt )
    txt = re.sub( r"\s+$", "", txt )
    return re.sub( r"\s+", " ", txt )

def whichQuadrant( x, y, quadrants ):
    angle = getAngle( x, y, quadrants["center_x"], quadrants["center_y"] )
    for i in range( len( quadrants["quadrants_arr"] ) ):
        if ( quadrants["quadrants_arr"][i]["min_angle"] <= angle ) and ( angle <= quadrants["quadrants_arr"][i]["max_angle"] ):
            return i
    return -1
    
def getQuadrants( widgets_arr ):
    text_arr = []
    quadrants = { "quadrants_arr" : [ { "min_angle" :  -90, "max_angle" :    0 },
                                      { "min_angle" :    0, "max_angle" :   90 },
                                      { "min_angle" :   90, "max_angle" :  180 },
                                      { "min_angle" : -179, "max_angle" :  -90 } ],
                  "center_x" : 0,
                  "center_y" : 0,
                  "min_r"    : 400,
                  "max_r"    : 0
                }
    # Collect text and set up coordinates using arrows
    for widget in widgets_arr:
        if widget["type"] == "text":
            text = removeExtraSpaces( removeHTML( widget["text"] ) )
            text_arr.append( { "text"   : text, 
                               "x"      : widget["x"], 
                               "y"      : widget["y"],
                               "height" : widget["height"],
                               "width"  : widget["width"] } )
        elif widget["type"] == "arrow":
            if widget["width"] > widget["height"]:
                quadrants["center_y"] = widget["y"]
                quadrants["max_r"] = 0.4 * widget["width"]
            else:
                quadrants["center_x"] = widget["x"]
    # Add text
    for txt in text_arr:
        index = whichQuadrant( txt["x"],  txt["y"], quadrants )
        quadrants["quadrants_arr"][index]["plan"] = txt
        #r = math.sqrt( ( txt["x"] - quadrants["center_x"] )**2 + ( txt["y"] - quadrants["center_y"] )**2 )
        #if r > quadrants["min_r"]:
        #    quadrants["min_r"] = r
    #quadrants["min_r"] = quadrants["min_r"] + 250
    return quadrants

In [28]:
g_quadrants = getQuadrants( g_widgets_arr )

In [ ]:
print( json.dumps( g_quadrants, indent=3 ) )

# Section C - Generate and post arguments

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-13_devils-advocate_01.png" width="50%" title="Image of a mural" />

<a id="step5"></a>
## Step 5: Generate arguments against the plans
For each plan, prompt a large language model to generate arguments against the plan using the IBM watsonx.ai foundation models Python library.

In [70]:
# If needed, experiment again with other models and different parameters

# g_model_id = "ibm/granite-13b-instruct-v1"

# g_prompt_parameters = {
#     "decoding_method": "sample",
#     "max_new_tokens": 200,
#     "min_new_tokens": 0,
#     "random_seed": 1683314208,
#     "stop_sequences": [ "\n\n" ],
#     "temperature": 0.7,
#     "top_k": 50,
#     "top_p": 1,
#     "repetition_penalty": 1.0
# }

In [30]:
for index in range( len( g_quadrants["quadrants_arr"] ) ):
    plan_text = g_quadrants["quadrants_arr"][index]["plan"]["text"]
    arguments_arr = listArgumentsAgainst( g_model_id, g_prompt_parameters, plan_text )
    if arguments_arr is None:
        print( "\nGenerating arguments against the plan in quadrant " + str( index ) + " failed." )
        break
    g_quadrants["quadrants_arr"][index]["arguments_arr"] = []
    for argument in arguments_arr:
        g_quadrants["quadrants_arr"][index]["arguments_arr"].append( { "text" : argument, "width" : 300, "height" : 250 } )

In [ ]:
print( json.dumps( g_quadrants, indent=3 ) )

<a id="step6"></a>
## Step 6: Post arguments in mural
- 6.1 Generate a random ( x, y ) position for each argument in the appropriate quadrant
- 6.2 Post arguments on sticky notes in the mural

### 6.1 Generate a random ( x, y ) position for each argument in the appropriate quadrant

In [32]:
from random import randrange

#g_quadrants["quadrants_arr"] = [ { "min_angle" :  (  -90 +  20 ), "max_angle" :  (   0 - 20 ) },
#                                 { "min_angle" :  (    0 +  20 ), "max_angle" :  (  90 - 20 ) },
#                                 { "min_angle" :  (   90 +  20 ), "max_angle" :  ( 180 - 20 ) },
#                                 { "min_angle" :  ( -179 +  20 ), "max_angle" :  ( -90 - 20 ) } ]

def randomPolar( min_r, max_r, min_angle, max_angle ):
    min_angle = min_angle + 10
    max_angle = max_angle - 10
    delta_r = max_r - min_r
    delta_angle = max_angle - min_angle
    r = min_r + ( 0.01 * randrange( 100 ) * delta_r )
    angle = min_angle + ( 0.01 * randrange( 100 ) * delta_angle )
    return r, angle

def xyFromPolar( r, angle ):
    x = r * math.cos( math.radians( angle ) )
    y = r * math.sin( math.radians( angle ) )
    x += g_quadrants["center_x"]
    y += g_quadrants["center_y"]
    return round( x ), round( y )

def randomPosition( quadrant_index ):
    min_r = g_quadrants["min_r"]
    max_r = g_quadrants["max_r"]
    min_angle = g_quadrants["quadrants_arr"][quadrant_index]["min_angle"]
    max_angle = g_quadrants["quadrants_arr"][quadrant_index]["max_angle"]
    r, angle = randomPolar( min_r, max_r, min_angle, max_angle )
    x, y = xyFromPolar( r, angle )
    return x, y

In [33]:
def rectanglesOverlap_org( left, right, top, bottom, obj ):
    existing_left = obj["x"]
    existing_right = existing_left + obj["width"]
    existing_top = obj["y"]
    existing_bottom = existing_top + obj["height"]
    if ( ( right > existing_left ) and ( left < existing_right ) and ( bottom > existing_top ) and ( top < existing_bottom ) ) or \
       ( ( right > existing_left ) and ( left < existing_right ) and ( bottom > existing_top ) and ( bottom < existing_bottom ) ) or \
       ( ( right > existing_left ) and ( left < existing_right ) and ( top > existing_top ) and ( top < existing_bottom ) ) or \
       ( ( right > existing_left ) and ( left < existing_right ) and ( bottom > existing_top ) and ( bottom < existing_bottom ) ):
        return True
    return False

def rectanglesOverlap( left, right, top, bottom, obj ):
    existing_left = obj["x"]
    existing_right = existing_left + obj["width"]
    existing_top = obj["y"]
    existing_bottom = existing_top + obj["height"]
    if ( right > existing_left ) and ( left < existing_right ) and ( bottom > existing_top ) and ( top < existing_bottom ):
        return True
    return False

def overlaps( argument_index_in, quadrant_index_in, x, y ):
    left = x
    right = x + 300
    top = y
    bottom = y + 250
    for quadrant_index in range( len( g_quadrants["quadrants_arr"] ) ):
        quadrant = g_quadrants["quadrants_arr"][quadrant_index]
        if rectanglesOverlap( left, right, top, bottom, quadrant["plan"] ):
            return True
        for argument_index in range( len( quadrant["arguments_arr"] ) ):
            argument = quadrant["arguments_arr"][argument_index]
            if ( ( argument_index != argument_index_in ) or ( quadrant_index != quadrant_index_in ) ) and ( "x" in argument ) and ( "y" in argument ):
                if rectanglesOverlap( left, right, top, bottom, argument ):
                    return True
    return False

In [34]:
for argument_index in range( 4 ):
    for quadrant_index in range( len( g_quadrants["quadrants_arr"] ) ):
        x, y = randomPosition( quadrant_index )
        while overlaps( argument_index, quadrant_index, x, y ):
            x, y = randomPosition( quadrant_index )
        g_quadrants["quadrants_arr"][quadrant_index]["arguments_arr"][argument_index]["x"] = x
        g_quadrants["quadrants_arr"][quadrant_index]["arguments_arr"][argument_index]["y"] = y

In [ ]:
print( json.dumps( g_quadrants, indent=3 ) )

### 6.2 Post arguments on sticky notes in the mural

In [40]:
# https://en.wikipedia.org/wiki/Web_colors

g_colors = [ [ "#FFB6C1FF",    # LightPink
               "#FF1493FF",    # DeepPink
               "#800080FF",    # Purple
               "#9932CCFF" ],  # DarkOrchid
             [ "#00FF00FF",    # Lime
               "#6B8E23FF",    # OliveDrab
               "#006400FF",    # DarkGreen
               "#90EE90FF" ],  # LightGreen
             [ "#00008BFF",    # DarkBlue
               "#0000FFFF",    # Blue
               "#1E90FFFF",    # DodgerBlue
               "#B0E0E6FF" ],  # PowderBlue
             [ "#FF4500FF",    # OrangeRed
               "#FFA500FF",    # Orange
               "#FFFF00FF",    # Yellow
               "#FFE4B5FF" ] ] # Moccasin

In [41]:
def addStickyToMural( mural_id, auth_token, sticky_data ):
    # https://developers.mural.co/public/reference/createstickynote
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/sticky-note"
    headers = { "Content-Type" : "application/json", "Accept" : "vnd.mural.preview", "Authorization" : "Bearer " + auth_token }
    data = json.dumps( sticky_data )
    response = requests.request( "POST", url, headers=headers, data=data )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )

In [42]:
def placeArgumentStickies( mural_id, auth_token, b_debug=False ):
    stickies_arr = []
    colors = g_colors.copy()
    for argument_index in range( 4 ):
        for quadrant_index in range( len( g_quadrants["quadrants_arr"] ) ):
            quadrant = g_quadrants["quadrants_arr"][quadrant_index]
            argument = quadrant["arguments_arr"][argument_index]
            x = argument["x"]
            y = argument["y"]
            text = argument["text"]
            bg_color = g_colors[quadrant_index][argument_index]
            sticky = { "shape"    : "rectangle", 
                       "style"    : { "backgroundColor" : bg_color, "fontSize" : 32 }, 
                       "x"        : x, 
                       "y"        : y, 
                       "height"   : 250,
                       "width"    : 300,
                       "text"     : text }
            addStickyToMural( mural_id, auth_token, sticky )
            stickies_arr.append( sticky )
    if b_debug:
        print( json.dumps( stickies_arr, indent=3 ) )

In [44]:
import time

time.sleep(5)

# Quick!  After running this cell, switch to your browser 
# tab where the mural is to see the stickies get added
# ...

placeArgumentStickies( g_mural_id, g_auth_token )

Something like this:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-13_devils-advocate.gif" width="50%" title="Image of a mural" />